In [1]:
import os
from pathlib import Path
import sys
node_type = os.getenv('BB_CPU')
venv_dir = f'/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-{node_type}'
venv_site_pkgs = Path(venv_dir) / 'lib' / f'python{sys.version_info.major}.{sys.version_info.minor}' / 'site-packages'
if venv_site_pkgs.exists():
    sys.path.insert(0, str(venv_site_pkgs))
    print(f"Added path '{venv_site_pkgs}' at start of search paths.")
else:
    print(f"Path '{venv_site_pkgs}' not found. Check that it exists and/or that it exists for node-type '{node_type}'.")

!pwd

%load_ext autoreload
%autoreload 2

Added path '/rds/homes/g/gaddcz/Projects/CPRD/virtual-envTorch2.0-icelake/lib/python3.10/site-packages' at start of search paths.
/rds/homes/g/gaddcz/Projects/CPRD/examples/data


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
import pickle
from hydra import compose, initialize
from omegaconf import OmegaConf
from CPRD.examples.modelling.SurvStreamGPT.experiment import run
from CPRD.data.foundational_loader import FoundationalDataModule
from CPRD.src.models.survival.task_heads.causal import SurvStreamGPTForCausalModelling

import time
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import os
import polars as pl
pl.Config.set_tbl_rows(10000)
import pandas as pd
pd.options.display.max_rows = 10000

torch.manual_seed(1337)
torch.set_float32_matmul_precision('medium')

logging.basicConfig(level=logging.INFO)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = "cpu"    # if more informative debugging statements are needed
print(f"Using device: {device}.")

path_to_dataset = "/rds/projects/g/gokhalkm-optimal/OPTIMAL_MASTER_DATASET/data/polars/split=train/"

Using device: cuda.


# check how fast PyArrow/Dask is in comparison

In [ ]:

# start_time = time.time()
# dataset = ds.dataset(path_to_dataset, format="parquet", partitioning="hive")
# print(f"Time to create dataset: {time.time() - start_time:.1f} seconds")

# start_time = time.time()
# dataset = pq.ParquetDataset(path_to_dataset, validate_schema=False)
# print(f"Time to create dataset: {time.time() - start_time:.1f} seconds")

import dask.dataframe as dd
# import pyarrow
# schema = pyarrow.schema([
#     ('HEALTH_AUTH', pyarrow.string())  # Replace with the correct type of HEALTH_AUTH
# ])
start_time = time.time()
dataset = dd.read_parquet(path_to_dataset, engine='pyarrow')
print(f"Time to create dataset: {time.time() - start_time:.1f} seconds")


# print(dataset.schema.to_string(show_field_metadata=False))

In [ ]:
# ds.dataset (10s of seconds)
start_time = time.time()
display(dataset.take([1.5e6]).to_pandas())
print(f"Time to sample: {time.time() - start_time:.4f} seconds")

# predictate filtering (0.5 seconds)
for index in range(100, 10000, 1002):
    chunk = int(index / 500)
    expression = ( ds.field('CHUNK') == chunk ) & (ds.field('row_nr') == index)
    
    start_time = time.time()
    table = dataset.to_table(filter=expression).to_pandas()
    print(time.time() - start_time)

print(table)

In [ ]:
# dask
print(dataset.head())
# index = 12345
# chunk = int(index / 500)
filtered_ddf = dataset.loc[index]
result = filtered_ddf.compute()
print(result)